Disclaimer: Nothing herein is financial advice, and NOT a recommendation to trade real money. Many platforms exist for simulated trading (paper trading) which can be used for building and developing the methods discussed. Please use common sense and always first consult a professional before trading or investing.

<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/3-Practical/FinRL_PaperTrading_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a >

# Part 1: Install FinRL

In [1]:
## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.8 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:00
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...
Selecting previously unselected package libgl1-mesa-glx:amd64.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../l

In [1]:
!pip install exchange_calendars==3.6.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for exchange_calendars: filename=exchange_calendars-3.6.3-py3-none-any.whl size=182619 sha256=f79526fc23bb8c03a958b1f8aa226e4057b75eabc32dcae3922ea0d267c9328d
  Stored in directory: /root/.cache/pip/wheels/93/ca/8f/1e1c90cc79fb3ca9b5413ff58e3fccf3baf2182c994c6dfd37
Successfully built exchange_calendars
  Attempting uninstall: exchange_calendars
    Found existing installation: exchange-calendars 4.2.7
    Uninstalling exchange-calendars-4.2.7:
      Successfully uninstalled exchange-calendars-4.2.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
finrl 0.3.6 requires exchange-calendars<5,>=4, but you have exchange-calendars 3.6.3

In [2]:
!pip install pandas==1.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 80.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.1
    Uninstalling pandas-2.0.1:
      Successfully uninstalled pandas-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
finrl 0.3.6 requires exchange-calendars<5,>=4, but you have exchange-calendars 3.6.3 which is incompatible.


## Import related modules

In [1]:
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.meta.env_stock_trading.env_stock_papertrading import AlpacaPaperTrading
from finrl.meta.data_processor import DataProcessor

from finrl.agents.stablebaselines3.models import DRLAgent

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl import config

import numpy as np
import pandas as pd
from pathlib import Path

import datetime
from pandas.tseries.offsets import BDay  # BDay is business day, not birthday..

/usr/local/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


## Import Dow Jones 30 Symbols

In [2]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)

## Get the API Keys Ready

In [3]:
API_KEY = "PKO361BYCXVIYHB1FCNX"
API_SECRET = "HCyg7FQNA3nlXRSNp2emmbqVR0GYw2iDWIO97h4w"
API_BASE_URL = 'https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'
env = StockTradingEnv

In [4]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [65]:
from finrl.meta.data_processor import DataProcessor
def get_data(start_date, end_date, if_vix = True):
      DP = DataProcessor(data_source = 'alpaca',
                    API_KEY = API_KEY, 
                    API_SECRET = API_SECRET, 
                    API_BASE_URL = API_BASE_URL
                    )
      print('Downloading ticker data')
      df = DP.download_data(start_date = start_date, 
                            end_date = end_date,
                            ticker_list = ticker_list, 
                            time_interval= '1Min')
      data = DP.clean_data(df)
      data = DP.add_technical_indicator(data, INDICATORS)
      data = DP.add_vix(data)
      data = data.fillna(0)
      return DP.df_to_array(data, if_vix)


In [164]:
today = datetime.datetime.today()

TEST_END_DATE = (today - BDay(1)).to_pydatetime().date()
TEST_START_DATE = (TEST_END_DATE - BDay(1)).to_pydatetime().date()
TRAIN_END_DATE = (TEST_START_DATE - BDay(1)).to_pydatetime().date()
TRAIN_START_DATE = (TRAIN_END_DATE - BDay(5)).to_pydatetime().date()
TRAINFULL_START_DATE = TRAIN_START_DATE
TRAINFULL_END_DATE = TEST_END_DATE

TRAIN_START_DATE = str(TRAIN_START_DATE)
TRAIN_END_DATE = str(TRAIN_END_DATE)
TEST_START_DATE = str(TEST_START_DATE)
TEST_END_DATE = str(TEST_END_DATE)
TRAINFULL_START_DATE = str(TRAINFULL_START_DATE)
TRAINFULL_END_DATE = str(TRAINFULL_END_DATE)

print("TRAIN_START_DATE : ", TRAIN_START_DATE)
print("TRAIN_END_DATE.  : ", TRAIN_END_DATE)
print("TEST_START_DATE  : ", TEST_START_DATE)
print("TEST_END_DATE    : ", TEST_END_DATE)

TRAIN_START_DATE :  2023-05-02
TRAIN_END_DATE.  :  2023-05-09
TEST_START_DATE  :  2023-05-10
TEST_END_DATE    :  2023-05-11


In [164]:
from __future__ import annotations

import gymnasium as gym
import numpy as np
from numpy import random as rd


class StockTradingEnv(gym.Env):
    def __init__(
        self,
        config,
        initial_account=50000,
        gamma=0.99,
        turbulence_thresh=99,
        min_stock_rate=0.1,
        max_stock=30,
        initial_capital=50000,
        buy_cost_pct=1e-3,
        sell_cost_pct=1e-3,
        reward_scaling=2**-11,
        initial_stocks=None,
    ):
        price_ary = config["price_array"]
        tech_ary = config["tech_array"]
        turbulence_ary = config["turbulence_array"]
        if_train = config["if_train"]
        
        self.price_ary = price_ary.astype(np.float32)
        self.tech_ary = tech_ary.astype(np.float32)
        self.turbulence_ary = turbulence_ary

        self.tech_ary = self.tech_ary * 2**-7
        self.turbulence_bool = (turbulence_ary > turbulence_thresh).astype(np.float32)
        self.turbulence_ary = (self.sigmoid_sign(turbulence_ary, turbulence_thresh) * 2**-5).astype(np.float32)

        stock_dim = self.price_ary.shape[1]
        self.gamma = gamma
        self.max_stock = max_stock
        self.min_stock_rate = min_stock_rate
        self.buy_cost_pct = buy_cost_pct
        self.sell_cost_pct = sell_cost_pct
        self.reward_scaling = reward_scaling
        self.initial_capital = initial_capital
        self.initial_stocks = (
            np.zeros(stock_dim, dtype=np.float32)
            if initial_stocks is None
            else initial_stocks
        )

        # reset()
        self.day = None
        self.amount = None
        self.stocks = None
        self.total_asset = None
        self.gamma_reward = None
        self.initial_total_asset = None

        # environment information
        self.env_name = "StockEnv"
        # self.state_dim = 1 + 2 + 2 * stock_dim + self.tech_ary.shape[1]
        # # amount + (turbulence, turbulence_bool) + (price, stock) * stock_dim + tech_dim
        self.state_dim = 1 + 2 + 3 * stock_dim + self.tech_ary.shape[1]
        # amount + (turbulence, turbulence_bool) + (price, stock) * stock_dim + tech_dim
        self.stocks_cd = None
        self.action_dim = stock_dim
        self.max_step = self.price_ary.shape[0] - 1
        self.if_train = if_train
        self.if_discrete = False
        self.target_return = 10.0
        self.episode_return = 0.0

        self.observation_space = gym.spaces.Box(
            low=-3000, high=3000, shape=(self.state_dim,), dtype=np.float32
        )
        self.action_space = gym.spaces.Box(
            low=-1, high=1, shape=(self.action_dim,), dtype=np.float32
        )

    def reset(self):
        self.day = 0
        price = self.price_ary[self.day]

        if self.if_train:
            self.stocks = (
                self.initial_stocks + rd.randint(0, 64, size=self.initial_stocks.shape)
            ).astype(np.float32)
            self.stocks_cool_down = np.zeros_like(self.stocks)
            self.amount = (
                self.initial_capital * rd.uniform(0.95, 1.05)
                - (self.stocks * price).sum()
            )
        else:
            self.stocks = self.initial_stocks.astype(np.float32)
            self.stocks_cool_down = np.zeros_like(self.stocks)
            self.amount = self.initial_capital

        self.total_asset = self.amount + (self.stocks * price).sum()
        self.initial_total_asset = self.total_asset
        self.gamma_reward = 0.0
        return self.get_state(price), {}  # state

    def step(self, actions):
        actions = (actions * self.max_stock).astype(int)

        self.day += 1
        price = self.price_ary[self.day]
        self.stocks_cool_down += 1

        if self.turbulence_bool[self.day] == 0:
            min_action = int(self.max_stock * self.min_stock_rate)  # stock_cd
            for index in np.where(actions < -min_action)[0]:  # sell_index:
                if price[index] > 0:  # Sell only if current asset is > 0
                    sell_num_shares = min(self.stocks[index], -actions[index])
                    self.stocks[index] -= sell_num_shares
                    self.amount += (
                        price[index] * sell_num_shares * (1 - self.sell_cost_pct)
                    )
                    self.stocks_cool_down[index] = 0
            for index in np.where(actions > min_action)[0]:  # buy_index:
                if (
                    price[index] > 0
                ):  # Buy only if the price is > 0 (no missing data in this particular date)
                    buy_num_shares = min(self.amount // price[index], actions[index])
                    self.stocks[index] += buy_num_shares
                    self.amount -= (
                        price[index] * buy_num_shares * (1 + self.buy_cost_pct)
                    )
                    self.stocks_cool_down[index] = 0

        else:  # sell all when turbulence
            self.amount += (self.stocks * price).sum() * (1 - self.sell_cost_pct)
            self.stocks[:] = 0
            self.stocks_cool_down[:] = 0

        state = self.get_state(price)
        total_asset = self.amount + (self.stocks * price).sum()
        reward = (total_asset - self.total_asset) * self.reward_scaling
        self.total_asset = total_asset

        self.gamma_reward = self.gamma_reward * self.gamma + reward
        done = self.day == self.max_step
        if done:
            reward = self.gamma_reward
            self.episode_return = total_asset / self.initial_total_asset

        return state, reward, done, False, dict()

    def get_state(self, price):
        amount = np.array(self.amount * (2**-12), dtype=np.float32)
        scale = np.array(2**-6, dtype=np.float32)
        return np.hstack(
            (
                amount,
                self.turbulence_ary[self.day],
                self.turbulence_bool[self.day],
                price * scale,
                self.stocks * scale,
                self.stocks_cool_down,
                self.tech_ary[self.day],
            )
        )  # state.astype(np.float32)

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh

## Calculate the DRL state dimension manually for paper trading

In [165]:
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim

# Creating Environment

In [186]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim
if_vix = True

# Train Data
train_price_array, train_tech_array, train_turbulence_array = get_data(TRAIN_START_DATE, TRAIN_END_DATE, if_vix = True)

# Test Data
test_price_array, test_tech_array, test_turbulence_array = get_data(TEST_START_DATE, TEST_END_DATE, if_vix = True)

Alpaca successfully connected


/usr/local/lib/python3.10/site-packages/finrl/meta/data_processors/processor_alpaca.py:52: FutureWarning: Timedelta.delta is deprecated and will be removed in a future version.
  if pd.Timedelta(time_interval).delta < day_delta:
/usr/local/lib/python3.10/site-packages/finrl/meta/data_processors/processor_alpaca.py:52: FutureWarning: Timedelta.delta is deprecated and will be removed in a future version.
  if pd.Timedelta(time_interval).delta < day_delta:


Alpaca successfully connected


/usr/local/lib/python3.10/site-packages/finrl/meta/data_processors/processor_alpaca.py:52: FutureWarning: Timedelta.delta is deprecated and will be removed in a future version.
  if pd.Timedelta(time_interval).delta < day_delta:
/usr/local/lib/python3.10/site-packages/finrl/meta/data_processors/processor_alpaca.py:52: FutureWarning: Timedelta.delta is deprecated and will be removed in a future version.
  if pd.Timedelta(time_interval).delta < day_delta:


In [187]:
count = 0
for i in range(len(train_turbulence_array)):
  count +=1
  if count ==390:
    count = 0
    #train_turbulence_array[i] = 100

count =0
for i in range(len(test_turbulence_array)):
  count +=1
  if count ==390:
    count = 0  
    #test_turbulence_array[i] = 100

train_env_config = {
    "price_array": train_price_array,
    "tech_array": train_tech_array,
    "turbulence_array": train_turbulence_array,
    "if_train": True,
}
env = StockTradingEnv
env_train = StockTradingEnv(config=train_env_config) 


test_env_config = {
        "price_array": test_price_array,
        "tech_array": test_tech_array,
        "turbulence_array": test_turbulence_array,
        "if_train": False,
    }
env_test = StockTradingEnv(config=test_env_config)

In [188]:
test_turbulence_array[-1]

7.525

In [189]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")
trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name="ddpg", total_timesteps=5000)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [190]:
environment = env_test
state = environment.reset()[0]
env_train.max_step
episode_returns = []  # the cumulative_return / initial_account
episode_total_assets = [environment.initial_total_asset]
done = False
count = 0
while not done:
  action = trained_ddpg.predict(state,  deterministic=True)[0]
  state, reward, done, _,_= environment.step(action)
  total_asset = (
                    environment.amount
                    + (
                        environment.price_ary[environment.day] * environment.stocks
                    ).sum()
                )
  episode_total_assets.append(total_asset)
  episode_return = total_asset / environment.initial_total_asset
  episode_returns.append(episode_return)
  count +=1
  if done:
      break

print("end_asset ", round(episode_total_assets[-1]))      
print("episode_return", episode_return)
print("Test Finished!")


end_asset  996227
episode_return 0.9962271483790472
Test Finished!


In [191]:
# ## install required packages

# if True:
#     # installing packages
#     !pip install swig
#     !pip install wrds
#     !pip install pyportfolioopt
#     !pip install trading_calendars
#     !pip install alpaca_trade_api
#     !pip install ccxt
#     !pip install jqdatasdk

#     !pip install lz4
#     !pip install tensorboardX
#     !pip install gputil
#     !pip install pyfolio-reloaded  #original pyfolio no longer maintained
#     !pip install optuna
#     !pip install plotly
#     !pip install ipywidgets
#     !pip install -U kaleido

In [192]:
import optuna

In [193]:
# Set Variables
## Fixed
tpm_hist = {}  # record tp metric values for trials
tp_metric = 'avgwl'  # specified trade_param_metric: ratio avg value win/loss
## Settable by User
n_trials = 100  # number of HP optimization runs
total_timesteps = 2000 # per HP optimization run
## Logging callback params
lc_threshold=3e-5
lc_patience=15
lc_trial_number=10

In [220]:
def sample_ddpg_params_all(trial:optuna.Trial,
                           # fixed values from previous study
                           learning_rate=0.0103,
                           batch_size=128,
                           buffer_size=int(1e6)):

    buffer_size = trial.suggest_int("buffer_size", int(1e4), int(1e6))
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1)
    batch_size = trial.suggest_int("batch_size", 32, 1024)
    gamma = trial.suggest_float("gamma", 0.1, 2)
    
    # Polyak coeff
    tau = trial.suggest_float("tau", 1e-5, 1)
    
    train_freq = trial.suggest_int("train_freq", 64, 2048)
    gradient_steps = trial.suggest_int("gradient_steps", 64, 2048)
    
    #noise_type = trial.suggest_categorical("noise_type", ["ornstein-uhlenbeck", "normal", None])
    #noise_std = trial.suggest_categorical("noise_std", [.0,.1,.2,.3, .4, .5, .6,.8] )

    # NOTE: Add "verybig" to net_arch when tuning HER (see TD3)
    net_arch = trial.suggest_categorical("net_arch", ["verysmall","small","medium", "big", "verybig",
                                                      "same_verysmall","same_small","same_medium", "same_big", "same_verybig"])
    # activation_fn = trial.suggest_categorical('activation_fn', [nn.Tanh, nn.ReLU, nn.ELU, nn.LeakyReLU])

    net_arch = {
        "verysmall": [64, 32],
        "small": [128, 64],
        "medium": [256, 512],
        "big": [512, 1024],
        "verybig": [2028, 1024],
        "same_verysmall": [64, 64],
        "same_small": [128, 128],
        "same_medium": [256, 256],
        "same_big": [512, 512],
        "same_verybig": [1024, 1024],
    }[net_arch]
  
    hyperparams = {
        "batch_size": batch_size,
        "buffer_size": buffer_size,
        "gamma": gamma,
        "gradient_steps": gradient_steps,
        "learning_rate": learning_rate,
        "tau": tau,
        "train_freq": train_freq,
        #"noise_std": noise_std,
        #"noise_type": noise_type,
        
        "policy_kwargs": dict(net_arch=net_arch)
    }
    return hyperparams

In [221]:
class LoggingCallback:
    def __init__(self,threshold,trial_number,patience):
      '''
      threshold:int tolerance for increase in objective
      trial_number: int Prune after minimum number of trials
      patience: int patience for the threshold
      '''
      self.threshold = threshold
      self.trial_number  = trial_number
      self.patience = patience
      print(f'Callback threshold {self.threshold}, \
            trial_number {self.trial_number}, \
            patience {self.patience}')
      self.cb_list = [] #Trials list for which threshold is reached
    def __call__(self,study:optuna.study, frozen_trial:optuna.Trial):
      #Setting the best value in the current trial
      study.set_user_attr("previous_best_value", study.best_value)
      
      #Checking if the minimum number of trials have pass
      if frozen_trial.number >self.trial_number:
          previous_best_value = study.user_attrs.get("previous_best_value",None)
          #Checking if the previous and current objective values have the same sign
          if previous_best_value * study.best_value >=0:
              #Checking for the threshold condition
              if abs(previous_best_value-study.best_value) < self.threshold: 
                  self.cb_list.append(frozen_trial.number)
                  #If threshold is achieved for the patience amount of time
                  if len(self.cb_list)>self.patience:
                      print('The study stops now...')
                      print('With number',frozen_trial.number ,'and value ',frozen_trial.value)
                      print('The previous and current best values are {} and {} respectively'
                              .format(previous_best_value, study.best_value))
                      study.stop()

In [222]:
agent = DRLAgent(env = env_train)

In [223]:
from IPython.display import clear_output
import sys 

os.makedirs("models",exist_ok=True)

def objective(trial:optuna.Trial):
  #Trial will suggest a set of hyperparamters from the specified range

  # Optional to optimize larger set of parameters
  # hyperparameters = sample_ddpg_params_all(trial)
  
  # Optimize buffer size, batch size, learning rate
  hyperparameters = sample_ddpg_params_all(trial)
  
  print(f'Hyperparameters from objective: {hyperparameters.keys()}')
  policy_kwargs = None  # default
  if 'policy_kwargs' in hyperparameters.keys():
    policy_kwargs = hyperparameters['policy_kwargs']
    del hyperparameters['policy_kwargs']
    print(f'Policy keyword arguments {policy_kwargs}')
  
  model_ddpg = agent.get_model("ddpg",
                               policy_kwargs = policy_kwargs,
                               model_kwargs = hyperparameters )
  
  #You can increase it for better comparison
  trained_ddpg = agent.train_model(model=model_ddpg,
                                   tb_log_name="ddpg",
                                   total_timesteps=total_timesteps)
  trained_ddpg.save('models/ddpg_{}.pth'.format(trial.number))
  clear_output(wait=True)
  
  #For the given hyperparamters, determine the account value in the trading period
  environment = env_test
  state = environment.reset()[0]
  env_train.max_step
  episode_returns = []  # the cumulative_return / initial_account
  episode_total_assets = [environment.initial_total_asset]
  done = False
  count = 0
  while not done:
    action = trained_ddpg.predict(state,  deterministic=True)[0]
    state, reward, done, _,_= environment.step(action)
    total_asset = (environment.amount + (environment.price_ary[environment.day] * environment.stocks).sum())
    episode_total_assets.append(total_asset)
    episode_return = total_asset / environment.initial_total_asset
    episode_returns.append(episode_return)
    count +=1
    if done:
        break
  return episode_return

#Create a study object and specify the direction as 'maximize'
#As you want to maximize sharpe
#Pruner stops not promising iterations
#Use a pruner, else you will get error related to divergence of model
#You can also use Multivariate samplere
#sampler = optuna.samplers.TPESampler(multivarite=True,seed=42)
sampler = optuna.samplers.TPESampler()

study = optuna.create_study(study_name="ddpg_study",direction='maximize',
                            sampler = sampler, pruner=optuna.pruners.HyperbandPruner())

logging_callback = LoggingCallback(threshold=lc_threshold,
                                   patience=lc_patience,
                                   trial_number=lc_trial_number)
#You can increase the n_trials for a better search space scanning
study.optimize(objective, n_trials=n_trials,catch=(ValueError,),callbacks=[logging_callback])

[I 2023-05-12 21:17:05,668] A new study created in memory with name: ddpg_study


Callback threshold 3e-05,             trial_number 10,             patience 15
Hyperparameters from objective: dict_keys(['batch_size', 'buffer_size', 'gamma', 'gradient_steps', 'learning_rate', 'tau', 'train_freq', 'policy_kwargs'])
Policy keyword arguments {'net_arch': [512, 512]}
{'batch_size': 164, 'buffer_size': 619570, 'gamma': 0.266418419784302, 'gradient_steps': 847, 'learning_rate': 0.07509894666748007, 'tau': 0.20401634706527402, 'train_freq': 140}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


[W 2023-05-12 21:17:52,484] Trial 0 failed with parameters: {'buffer_size': 619570, 'learning_rate': 0.07509894666748007, 'batch_size': 164, 'gamma': 0.266418419784302, 'tau': 0.20401634706527402, 'train_freq': 140, 'gradient_steps': 847, 'net_arch': 'same_big'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-223-cd6f01d505e0>", line 27, in objective
    trained_ddpg = agent.train_model(model=model_ddpg,
  File "/usr/local/lib/python3.10/site-packages/finrl/agents/stablebaselines3/models.py", line 103, in train_model
    model = model.learn(
  File "/usr/local/lib/python3.10/site-packages/stable_baselines3/ddpg/ddpg.py", line 123, in learn
    return super().learn(
  File "/usr/local/lib/python3.10/site-packages/stable_baselines3/td3/td3.py", line 222, in learn
    return super().learn(
  Fil

KeyboardInterrupt: ignored

In [210]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [204]:
environment.amount

14.068179407005672

In [198]:
import joblib
joblib.dump(study, "final_ddpg_study__.pkl")

['final_ddpg_study__.pkl']

In [199]:
#Get the best hyperparamters
print('Hyperparameters after tuning',study.best_params)
print('Hyperparameters before tuning',config.DDPG_PARAMS)

Hyperparameters after tuning {'buffer_size': 404206, 'learning_rate': 0.017906980149605733, 'batch_size': 1021, 'gamma': 1.9889644863918345, 'tau': 0.0065353029388855455, 'train_freq': 1340, 'gradient_steps': 1396, 'net_arch': 'same_big'}
Hyperparameters before tuning {'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}


In [200]:
study.best_trial

FrozenTrial(number=10, state=TrialState.COMPLETE, values=[0.996522505679407], datetime_start=datetime.datetime(2023, 5, 12, 20, 37, 32, 963740), datetime_complete=datetime.datetime(2023, 5, 12, 20, 37, 56, 781820), params={'buffer_size': 404206, 'learning_rate': 0.017906980149605733, 'batch_size': 1021, 'gamma': 1.9889644863918345, 'tau': 0.0065353029388855455, 'train_freq': 1340, 'gradient_steps': 1396, 'net_arch': 'same_big'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'buffer_size': IntDistribution(high=1000000, log=False, low=10000, step=1), 'learning_rate': FloatDistribution(high=1.0, log=False, low=1e-05, step=None), 'batch_size': IntDistribution(high=1024, log=False, low=32, step=1), 'gamma': FloatDistribution(high=2.0, log=False, low=0.1, step=None), 'tau': FloatDistribution(high=1.0, log=False, low=1e-05, step=None), 'train_freq': IntDistribution(high=2048, log=False, low=64, step=1), 'gradient_steps': IntDistribution(high=2048, log=False, low=64, s

In [201]:
from stable_baselines3 import DDPG
tuned_model_ddpg = DDPG.load('models/ddpg_{}.pth'.format(study.best_trial.number),env=env_train)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [207]:

environment = env_test
state = environment.reset()[0]
env_train.max_step
episode_returns = []  # the cumulative_return / initial_account
episode_total_assets = [environment.initial_total_asset]
done = False
count = 0
while not done:
  action = tuned_model_ddpg.predict(state,  deterministic=True)[0]
  print(environment.amount)
  state, reward, done, _,_= environment.step(action)
  total_asset = (
                    environment.amount
                    + (
                        environment.price_ary[environment.day] * environment.stocks
                    ).sum()
                )
  #print(total_asset)
  episode_total_assets.append(total_asset)
  episode_return = total_asset / environment.initial_total_asset
  episode_returns.append(episode_return)
  count +=1
  if done:
      break
episode_return

1000000.0
734602.9579565047
469277.24800758355
203896.30137386307
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.068179407005672
14.

0.996522505679407

In [203]:
environment.stocks

array([400., 400.,   0., 400.,   0.,   0., 400., 400.,   0.,   0., 400.,
         0., 400., 400.,   0.,   0., 400.,   0.,   0.,   0.,   0., 400.,
         0., 389., 300.,   0., 300.,   0., 303., 300.], dtype=float32)

## Train

In [ ]:
ERL_PARAMS = {"learning_rate": 3e-6,"batch_size": 2048,"gamma":  0.985,
        "seed":312,"net_dimension":[128,64], "target_step":5000, "eval_gap":30,
        "eval_times":1} 
env = StockTradingEnv
#if you want to use larger datasets (change to longer period), and it raises error, 
#please try to increase "target_step". It should be larger than the episode steps. 

In [ ]:
train(start_date = '2022-08-25', 
      end_date = '2022-08-31',
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= '1Min', 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env,
      model_name='ppo',
      if_vix=True, 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl', #current_working_dir
      break_step=1e5)

Alpaca successfully connected


/usr/local/lib/python3.10/site-packages/finrl/meta/data_processors/processor_alpaca.py:52: FutureWarning: Timedelta.delta is deprecated and will be removed in a future version.
  if pd.Timedelta(time_interval).delta < day_delta:


The price of the first row for ticker  AAPL  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AMGN  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AXP  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  BA  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CAT  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CRM  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CSCO  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CVX  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  DIS  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  DOW  is NaN.  It will filled with the first valid price.
The price of the first row for ticker 

/usr/local/lib/python3.10/site-packages/finrl/meta/data_processors/processor_alpaca.py:52: FutureWarning: Timedelta.delta is deprecated and will be removed in a future version.
  if pd.Timedelta(time_interval).delta < day_delta:


The price of the first row for ticker  VIXY  is NaN.  It will filled with the first valid price.
Stock Dimension: 30, State Space: 301


AttributeError: ignored

## Test

In [ ]:
account_value_erl=test(start_date = '2022-09-01', 
                      end_date = '2022-09-02',
                      ticker_list = ticker_list, 
                      data_source = 'alpaca',
                      time_interval= '1Min', 
                      technical_indicator_list= INDICATORS,
                      drl_lib='elegantrl', 
                      env=env, 
                      model_name='ppo',
                      if_vix=True, 
                      API_KEY = API_KEY, 
                      API_SECRET = API_SECRET, 
                      API_BASE_URL = API_BASE_URL,
                      cwd='./papertrading_erl',
                      net_dimension = ERL_PARAMS['net_dimension'])

## Use full data to train 

After tuning well, retrain on the training and testing sets

In [ ]:
train(start_date = '2022-08-25', 
      end_date = '2022-09-02',
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= '1Min', 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env, 
      model_name='ppo',
      if_vix=True, 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl_retrain',
      break_step=2e5)

# Part 3: Deploy the agent

## Setup Alpaca Paper trading environment

In [ ]:
import datetime
import threading
from finrl.meta.data_processors.processor_alpaca import AlpacaProcessor
import alpaca_trade_api as tradeapi
import time
import pandas as pd
import numpy as np
import torch
import gym

class AlpacaPaperTrading():

    def __init__(self,ticker_list, time_interval, drl_lib, agent, cwd, net_dim, 
                 state_dim, action_dim, API_KEY, API_SECRET, 
                 API_BASE_URL, tech_indicator_list, turbulence_thresh=30, 
                 max_stock=1e2, latency = None):
        #load agent
        self.drl_lib = drl_lib
        if agent =='ppo':
            if drl_lib == 'elegantrl':              
                agent_class = AgentPPO
                agent = agent_class(net_dim, state_dim, action_dim)
                actor = agent.act
                # load agent
                try:  
                    cwd = cwd + '/actor.pth'
                    print(f"| load actor from: {cwd}")
                    actor.load_state_dict(torch.load(cwd, map_location=lambda storage, loc: storage))
                    self.act = actor
                    self.device = agent.device
                except BaseException:
                    raise ValueError("Fail to load agent!")
                        
            elif drl_lib == 'rllib':
                from ray.rllib.agents import ppo
                from ray.rllib.agents.ppo.ppo import PPOTrainer
                
                config = ppo.DEFAULT_CONFIG.copy()
                config['env'] = StockEnvEmpty
                config["log_level"] = "WARN"
                config['env_config'] = {'state_dim':state_dim,
                            'action_dim':action_dim,}
                trainer = PPOTrainer(env=StockEnvEmpty, config=config)
                trainer.restore(cwd)
                try:
                    trainer.restore(cwd)
                    self.agent = trainer
                    print("Restoring from checkpoint path", cwd)
                except:
                    raise ValueError('Fail to load agent!')
                    
            elif drl_lib == 'stable_baselines3':
                from stable_baselines3 import PPO
                
                try:
                    #load agent
                    self.model = PPO.load(cwd)
                    print("Successfully load model", cwd)
                except:
                    raise ValueError('Fail to load agent!')
                    
            else:
                raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
               
        else:
            raise ValueError('Agent input is NOT supported yet.')
            
            
            
        #connect to Alpaca trading API
        try:
            self.alpaca = tradeapi.REST(API_KEY,API_SECRET,API_BASE_URL, 'v2')
        except:
            raise ValueError('Fail to connect Alpaca. Please check account info and internet connection.')
        
        #read trading time interval
        if time_interval == '1s':
            self.time_interval = 1
        elif time_interval == '5s':
            self.time_interval = 5
        elif time_interval == '1Min':
            self.time_interval = 60
        elif time_interval == '5Min':
            self.time_interval = 60 * 5
        elif time_interval == '15Min':
            self.time_interval = 60 * 15
        else:
            raise ValueError('Time interval input is NOT supported yet.')
        
        #read trading settings
        self.tech_indicator_list = tech_indicator_list
        self.turbulence_thresh = turbulence_thresh
        self.max_stock = max_stock 
        
        #initialize account
        self.stocks = np.asarray([0] * len(ticker_list)) #stocks holding
        self.stocks_cd = np.zeros_like(self.stocks) 
        self.cash = None #cash record 
        self.stocks_df = pd.DataFrame(self.stocks, columns=['stocks'], index = ticker_list)
        self.asset_list = []
        self.price = np.asarray([0] * len(ticker_list))
        self.stockUniverse = ticker_list
        self.turbulence_bool = 0
        self.equities = []
        
    def test_latency(self, test_times = 10): 
        total_time = 0
        for i in range(0, test_times):
            time0 = time.time()
            self.get_state()
            time1 = time.time()
            temp_time = time1 - time0
            total_time += temp_time
        latency = total_time/test_times
        print('latency for data processing: ', latency)
        return latency
        
    def run(self):
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
          self.alpaca.cancel_order(order.id)
    
        # Wait for market to open.
        print("Waiting for market to open...")
        tAMO = threading.Thread(target=self.awaitMarketOpen)
        tAMO.start()
        tAMO.join()
        print("Market opened.")
        while True:

          # Figure out when the market will close so we can prepare to sell beforehand.
          clock = self.alpaca.get_clock()
          closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          self.timeToClose = closingTime - currTime
    
          if(self.timeToClose < (60)):
            # Close all positions when 1 minutes til market close.
            print("Market closing soon. Stop trading.")
            break
            
            '''# Close all positions when 1 minutes til market close.
            print("Market closing soon.  Closing positions.")
    
            positions = self.alpaca.list_positions()
            for position in positions:
              if(position.side == 'long'):
                orderSide = 'sell'
              else:
                orderSide = 'buy'
              qty = abs(int(float(position.qty)))
              respSO = []
              tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
              tSubmitOrder.start()
              tSubmitOrder.join()
    
            # Run script again after market close for next trading day.
            print("Sleeping until market close (15 minutes).")
            time.sleep(60 * 15)'''
            
          else:
            trade = threading.Thread(target=self.trade)
            trade.start()
            trade.join()
            last_equity = float(self.alpaca.get_account().last_equity)
            cur_time = time.time()
            self.equities.append([cur_time,last_equity])
            time.sleep(self.time_interval)
            
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while(not isOpen):
          clock = self.alpaca.get_clock()
          openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          timeToOpen = int((openingTime - currTime) / 60)
          print(str(timeToOpen) + " minutes til market open.")
          time.sleep(60)
          isOpen = self.alpaca.get_clock().is_open
    
    def trade(self):
        state = self.get_state()
        
        if self.drl_lib == 'elegantrl':
            with torch.no_grad():
                s_tensor = torch.as_tensor((state,), device=self.device)
                a_tensor = self.act(s_tensor)  
                action = a_tensor.detach().cpu().numpy()[0]  
            action = (action * self.max_stock).astype(int)
            
        elif self.drl_lib == 'rllib':
            action = self.agent.compute_single_action(state)
        
        elif self.drl_lib == 'stable_baselines3':
            action = self.model.predict(state)[0]
            
        else:
            raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
        
        self.stocks_cd += 1
        if self.turbulence_bool == 0:
            min_action = 10  # stock_cd
            for index in np.where(action < -min_action)[0]:  # sell_index:
                sell_num_shares = min(self.stocks[index], -action[index])
                qty =  abs(int(sell_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'sell', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for index in np.where(action > min_action)[0]:  # buy_index:
                if self.cash < 0:
                    tmp_cash = 0
                else:
                    tmp_cash = self.cash
                buy_num_shares = min(tmp_cash // self.price[index], abs(int(action[index])))
                if (buy_num_shares != buy_num_shares): # if buy_num_change = nan
                    qty = 0 # set to 0 quantity
                else:
                    qty = abs(int(buy_num_shares))
                qty = abs(int(buy_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'buy', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0
                
        else:  # sell all when turbulence
            positions = self.alpaca.list_positions()
            for position in positions:
                if(position.side == 'long'):
                    orderSide = 'sell'
                else:
                    orderSide = 'buy'
                qty = abs(int(float(position.qty)))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
            
            self.stocks_cd[:] = 0
            
    
    def get_state(self):
        alpaca = AlpacaProcessor(api=self.alpaca)
        price, tech, turbulence = alpaca.fetch_latest_data(ticker_list = self.stockUniverse, time_interval='1Min',
                                                     tech_indicator_list=self.tech_indicator_list)
        turbulence_bool = 1 if turbulence >= self.turbulence_thresh else 0
        
        turbulence = (self.sigmoid_sign(turbulence, self.turbulence_thresh) * 2 ** -5).astype(np.float32)
        
        tech = tech * 2 ** -7
        positions = self.alpaca.list_positions()
        stocks = [0] * len(self.stockUniverse)
        for position in positions:
            ind = self.stockUniverse.index(position.symbol)
            stocks[ind] = ( abs(int(float(position.qty))))
        
        stocks = np.asarray(stocks, dtype = float)
        cash = float(self.alpaca.get_account().cash)
        self.cash = cash
        self.stocks = stocks
        self.turbulence_bool = turbulence_bool 
        self.price = price
        
        
        
        amount = np.array(self.cash * (2 ** -12), dtype=np.float32)
        scale = np.array(2 ** -6, dtype=np.float32)
        state = np.hstack((amount,
                    turbulence,
                    self.turbulence_bool,
                    price * scale,
                    self.stocks * scale,
                    self.stocks_cd,
                    tech,
                    )).astype(np.float32)
        state[np.isnan(state)] = 0.0
        state[np.isinf(state)] = 0.0
        print(len(self.stockUniverse))
        return state
        
    def submitOrder(self, qty, stock, side, resp):
        if(qty > 0):
          try:
            self.alpaca.submit_order(stock, qty, side, "market", "day")
            print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
            resp.append(True)
          except:
            print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
            resp.append(False)
        else:
          print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
          resp.append(True)

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh
    
class StockEnvEmpty(gym.Env):
    #Empty Env used for loading rllib agent
    def __init__(self,config):
      state_dim = config['state_dim']
      action_dim = config['action_dim']
      self.env_num = 1
      self.max_step = 10000
      self.env_name = 'StockEnvEmpty'
      self.state_dim = state_dim  
      self.action_dim = action_dim
      self.if_discrete = False  
      self.target_return = 9999
      self.observation_space = gym.spaces.Box(low=-3000, high=3000, shape=(state_dim,), dtype=np.float32)
      self.action_space = gym.spaces.Box(low=-1, high=1, shape=(action_dim,), dtype=np.float32)
        
    def reset(self):
        return 

    def step(self, actions):
        return

## Run Paper trading

In [ ]:
print(DOW_30_TICKER)

In [ ]:
state_dim

In [ ]:
action_dim

In [ ]:
paper_trading_erl = AlpacaPaperTrading(ticker_list = DOW_30_TICKER, 
                                       time_interval = '1Min', 
                                       drl_lib = 'elegantrl', 
                                       agent = 'ppo', 
                                       cwd = './papertrading_erl_retrain', 
                                       net_dim = ERL_PARAMS['net_dimension'], 
                                       state_dim = state_dim, 
                                       action_dim= action_dim, 
                                       API_KEY = API_KEY, 
                                       API_SECRET = API_SECRET, 
                                       API_BASE_URL = API_BASE_URL, 
                                       tech_indicator_list = INDICATORS, 
                                       turbulence_thresh=30, 
                                       max_stock=1e2)
paper_trading_erl.run()

# Part 4: Check Portfolio Performance

In [ ]:
import alpaca_trade_api as tradeapi
import exchange_calendars as tc
import numpy as np
import pandas as pd
import pytz
import yfinance as yf
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from datetime import datetime as dt
from finrl.plot import backtest_stats
import matplotlib.pyplot as plt

In [ ]:
def get_trading_days(start, end):
    nyse = tc.get_calendar('NYSE')
    df = nyse.sessions_in_range(pd.Timestamp(start,tz=pytz.UTC),
                                pd.Timestamp(end,tz=pytz.UTC))
    trading_days = []
    for day in df:
        trading_days.append(str(day)[:10])

    return trading_days

def alpaca_history(key, secret, url, start, end):
    api = tradeapi.REST(key, secret, url, 'v2')
    trading_days = get_trading_days(start, end)
    df = pd.DataFrame()
    for day in trading_days:
        df = df.append(api.get_portfolio_history(date_start = day,timeframe='5Min').df.iloc[:78])
    equities = df.equity.values
    cumu_returns = equities/equities[0]
    cumu_returns = cumu_returns[~np.isnan(cumu_returns)]
    
    return df, cumu_returns

def DIA_history(start):
    data_df = yf.download(['^DJI'],start=start, interval="5m")
    data_df = data_df.iloc[:]
    baseline_returns = data_df['Adj Close'].values/data_df['Adj Close'].values[0]
    return data_df, baseline_returns

## Get cumulative return

In [ ]:
API_KEY = ""
API_SECRET = ""
API_BASE_URL = 'https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'

In [ ]:
df_erl, cumu_erl = alpaca_history(key=API_KEY, 
                                  secret=API_SECRET, 
                                  url=API_BASE_URL, 
                                  start='2022-09-01', #must be within 1 month
                                  end='2022-09-12') #change the date if error occurs


In [ ]:
df_djia, cumu_djia = DIA_history(start='2022-09-01')

In [ ]:
df_erl.tail()

In [ ]:
returns_erl = cumu_erl -1 
returns_dia = cumu_djia - 1
returns_dia = returns_dia[:returns_erl.shape[0]]
print('len of erl return: ', returns_erl.shape[0])
print('len of dia return: ', returns_dia.shape[0])

In [ ]:
returns_erl

## plot and save

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=1000)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Stock Trading (Paper trading)', fontsize=20)
plt.plot(returns_erl, label = 'ElegantRL Agent', color = 'red')
#plt.plot(returns_sb3, label = 'Stable-Baselines3 Agent', color = 'blue' )
#plt.plot(returns_rllib, label = 'RLlib Agent', color = 'green')
plt.plot(returns_dia, label = 'DJIA', color = 'grey')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Year 2021', fontsize=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(78))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(['','10-19','','10-20',
                                                    '','10-21','','10-22']))
plt.legend(fontsize=10.5)
plt.savefig('papertrading_stock.png')